In [1]:
# YOLO model
from ultralytics import YOLO
model=YOLO('/work/marioeduardo-a/ftc/models/best-organizers.pt')
# model=YOLO('/work/marioeduardo-a/best.pt')
# model=YOLO('best-yulun.pt')

In [2]:
# Tracker config file
import os
max_frames=50

tracker_type = 'bytetrack'
track_high_thresh = 0.3
track_low_thresh = 0.05
new_track_thresh = 0.2
track_buffer = 40
match_thresh = 0.5

TrackerName = f"Train{max_frames}"
trackers_folder = f"TrackEval/data/trackers/mot_challenge/FISH{max_frames}-train"
tracker_folder = os.path.join(trackers_folder, TrackerName)
result_folder = os.path.join(tracker_folder, "data")
tracker_config_file = os.path.join(tracker_folder, "custom-tracker.yaml")
raw_result_file = os.path.join(result_folder, "raw.txt")
result_file = os.path.join(result_folder, f"FISH{max_frames}.txt")
match_file = os.path.join(tracker_folder, f"FISH{max_frames}-pedestrian-bestmatch.txt")

if not os.path.exists(result_folder):
  os.makedirs(result_folder)

with open(tracker_config_file, 'w') as f:
  config = '# Ultralytics YOLO 🚀, AGPL-3.0 license\n' + \
        '# Default YOLO tracker settings for ByteTrack tracker https://github.com/ifzhang/ByteTrack\n' + \
        '\n' + \
        f'tracker_type: {tracker_type}  # tracker type, [\'botsort\', \'bytetrack\']\n' + \
        f'track_high_thresh: {track_high_thresh}  # threshold for the first association\n' + \
        f'track_low_thresh: {track_low_thresh}  # threshold for the second association\n' + \
        f'new_track_thresh: {new_track_thresh}  # threshold for init new track if the detection does not match any tracks\n' + \
        f'track_buffer: {track_buffer}  # buffer to calculate the time when to remove tracks\n' + \
        f'match_thresh: {match_thresh}  # threshold for matching tracks\n' + \
        '# min_box_area: 10  # threshold for min box areas(for tracker evaluation, not used for now)\n' + \
        '# mot20: False  # for tracker evaluation(not used for now)'
  f.write(config)

In [3]:
# Files for tracker
import os

FISH=f"TrackEval/data/gt/mot_challenge/FISH{max_frames}-train/FISH{max_frames}"
# FISH=f"TrackEval/data/gt/mot_challenge/FISH-train{max_frames}/FISH"

#gt folder
fn=os.path.join(FISH,"gt")
if not os.path.exists(fn):
    os.makedirs(fn)

#PerfectTracker folder
fn=f"TrackEval/data/trackers/mot_challenge/FISH{max_frames}-train/PerfectTracker/data"
if not os.path.exists(fn):
    os.makedirs(fn)

                # gt
# TrackEval/data/gt/mot_challenge/FISH-train/FISH/gt/gt.txt
fn=os.path.join(FISH,"gt/gt.txt")
with open("TrackEval/data/gt/mot_challenge/FISH-train/FISH/gt/gt.txt","r") as f:
    with open(fn,"w") as g:
        frames=0
        for line in f:
            g.write(line)
            frames+=1
            if frames> max_frames*10-1:
                break

#Perfect tracker

!cp {FISH}/gt/gt.txt "TrackEval/data/trackers/mot_challenge/FISH{max_frames}-train/PerfectTracker/data/FISH10.txt"

#TrackEval/data/gt/mot_challenge/FISH-train/FISH/seqinfo.ini
fn=os.path.join(FISH, "seqinfo.ini")
with open(fn,"w") as f:
   f.write("[Sequence]\n")
   f.write(f"name=FISH{max_frames}\n")
   f.write(f"seqLength={max_frames}")

# seqmaps
seqmaps= "TrackEval/data/gt/mot_challenge/seqmaps"
fns=[os.path.join(seqmaps, f'FISH{max_frames}-all.txt'),\
     os.path.join(seqmaps, f'FISH{max_frames}-test.txt'),\
     os.path.join(seqmaps, f'FISH{max_frames}-train.txt')]
for fn in fns:
    with open(fn,"w") as f:
        f.write("NAME\n")
        f.write(f"FISH{max_frames}")


In [ ]:
import cv2
#tracking the model and write the result to txt
video = "/work/marioeduardo-a/ftc/FTC-2024-data/Train/train.mp4"
cap = cv2.VideoCapture(video)
with open(raw_result_file, 'w') as r:
    frames = 1
    print("before")
    while cap.isOpened() and frames<=max_frames:
        success, frame = cap.read()
        print("frame:",frames)
        if success:
            
            print("before track")
            results = model.track(frame, persist=True, tracker=tracker_config_file)
            print("after track")
            

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                x -= w/2
                y -= h/2
                r.write(f'{frames} {track_id} {x:5.2f} {y:5.2f} {w:5.2f} {h:5.2f} -1 -1 -1 -1\n')
        else:
            # Break the loop if the end of the video is reached
            break
        frames += 1
# Release the video capture object and close the display window
# cap.close()
cap.release()
cv2.destroyAllWindows()

before
frame: 1
before track



In [3]:
cap

< cv2.VideoCapture 0x7f9adc673870>

In [4]:
cap.read()

(True,
 array([[[ 26,   0,   0],
         [ 23,   0,   0],
         [  7,   0,  24],
         ...,
         [  0,   9,   6],
         [  0,   7,   0],
         [  0,   0,   0]],
 
        [[ 23,   0,   0],
         [ 26,   0,   0],
         [  6,   0,  23],
         ...,
         [186, 206, 203],
         [182, 204, 180],
         [158, 180, 156]],
 
        [[ 11,   6,  18],
         [ 12,   7,  19],
         [126,   0,  90],
         ...,
         [189, 208, 207],
         [184, 206, 182],
         [160, 182, 158]],
 
        ...,
 
        [[  0,   2,   3],
         [116, 123, 124],
         [111, 118, 119],
         ...,
         [178, 191, 199],
         [178, 191, 199],
         [178, 191, 199]],
 
        [[  0,   1,   2],
         [  0,   1,   2],
         [  0,   1,   2],
         ...,
         [  0,   0,   4],
         [  0,   0,   4],
         [  0,   0,   4]],
 
        [[  0,   1,   2],
         [  0,   1,   2],
         [  0,   1,   2],
         ...,
         [  0,   0,  